<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/main/3-other-languages/Python-para-ciencia-de-datos/Sesion%204a%20-%20Estructuracion%20de%20datos%20de%20texto%20no%20estructurados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al Análisis y Mineria de Textos

Análisis y mineria de texto es el proceso de extraer información significativa a partir de datos textuales, revelando ideas que de otro modo permanecerían ocultas en grandes volumenes de texto. El termino "texto" aqui se refiere a cualquier conjunto de caracteres: podria desde libros enteros hasta una sola oracion o palabra.

Esta sesión y la siguiente son una **introducción** al análisis de texto. Cubriremos los siguientes temas:

1. Estructuracion de datos de texto no estructurados
    1. Reconocimiento de caracteres desde documentos en PDF
1. Limpieza y preparacion de datos de texto
    1. Expresiones regulares y patrones de caracteres en datos de texto  
    1. Preprocesamiento de datos textuales  
1. Analisis descriptivo de datos de textos
    1. Conteo de palabras
    1. Análisis de sentimiento  
1. Clasificación de textos

Veremos los dos primeros puntos en la sesion de hoy y los dos ultimos puntos manana.

Esta sesión asume conocimientos previos de Python y Pandas, así como cierto conocimiento de visualización de datos usando seaborn. Todo lo que cubrimos en las tres primeras sesiones de este taller es suficiente base para continuar con analisis de textos.

Usaremos las siguientes bibliotecas en este notebook:

- **pdfminer** para "leer" archivos PDF y extraer su contenido en textos
- **pandas** para operaciones con dataframes  
- **re** para expresiones regulares  
- **spacy** para procesamiento de datos textuales

# Sesion 4 - Estructuracion de datos de texto y preparacion de datos de texto

# 1. Estructuracion de datos de texto no estructurados

Los datos de en volumenes raramente tienen una estructura predeterminada. En muchos casos, estos vienen de archivos individuales que contienen textos. Por ejemplo, estos pueden ser un folder con decenas, cientos o miles de archivos en formato PDF, Word o `.txt`.

Dar una estructura a estos archivos requiere que evaluemos cual es la mejor forma que una tabla de datos para cierto volumen de textos puede adquirir. Por ejemplo, al trabajar con un folder con cientos de archivos PDF de texto, podemos estructurar la tabla para que tenga una fila por documento, una fila fila por parrafo, una fila por oracion o incluso una fila por palabra.

Para el siguiente ejemplo que usaremos en la primera mitad de esta sesion, partiremos de un volumen de documentos publicos del Banco Mundial. Estos documentos corresponden a algunos de los **reportes publicos que el Banco Mundial ha producido en Espanol sobre Republica Dominicana desde 2010 a 2024**.

Estos documentos estan en la carpeta `docs/` y son todos archivos en PDF. La imagen debajo es una captura de pantalla del contenido de la carpeta.

<img src="img/docs.png" width=800 />

## 1.1 Reconocimiento optico de caracteres desde documentos en PDF

El reconocimiento optico de caracteres (*Optical Character Recognition*--OCR por sus siglas en ingles) es una operacion comun en analisis de textos. Consiste en transformar textos que estan en algun documento (PDF or Word, por ejemplo) o en una imagen en un formato para texto que pueda ser operado por un lenguaje de programacion, por ejemplo en una string en Python.

En este ejemplo, usaremos el paquete `pdfminer` para reconocer los caracteres de estos archivos PDF. Ten en cuenta que tambien es posible reconocer textos de archivos en Word o de imagenes a strings en Python.

- [Mira aqui](https://github.com/microsoft/Simplify-Docx) un ejemplo (en ingles) para transformar archivos de Word a texto en Python
- [Aqui](https://medium.com/do-it-with-code/extract-text-from-images-using-python-ocr-dc7092adf9a8) un ejemplo (tambien en ingles) para transformar imagenes con textos a strings en Python

En general, existen muchas soluciones con metodos que logran resultados aceptables o buenos para reconocer caracteres de documentos o caracteres producidos con computadoras. Sin embargo, **reconocer escritura a mano**  es un proceso mucho mas complicado para el cual no existen soluciones gratuitas predeterminadas que funcionen bien en todos los casos. El ejemplo que veremos en esta sesion y los dos links del parrafo anterior posiblemente no logren buenos resultados para caracteres escritos a mano.

Ahora continuaremos importando los paquetes necesarios para "leer" un PDF a texto en Python:

In [ ]:
# Instala PDF miner si no lo tienes
# !pip install pdfminer

# Modulos de pdfminer para leer PDF
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

# Paquetes para trabajar con directorios
import os
import io

Empezaremos creando una lista con la ubicacion de todos los documentos que queremos leer desde PDF. Estos estan en la carpeta `doc/`. Para esto, vamos a usar el paquete `os` que nos permite interactuar con folderes para trabajar con archivos.

In [ ]:
# Folder cuyos archivos queremos incluir en la lista
folder = 'docs/'

# Definiendo una lista vacia para agregar la ruta de los archivos PDF
docs = []

El siguiente loop explora todos los archivos en `folder` y anade a la lista `docs` aquellos que tienen la extension `.pdf`:

In [ ]:
# Bucle a traves de los archivos en "folder"
for archivo in os.listdir(folder):
    
    if archivo.endswith(".pdf"):             # si el archivo es un PDF, continuamos
        
        doc = os.path.join(folder, archivo)  # os.path.join une un nombre de carpeta y archivo para dar una ruta completa
        print(f'Documento: {doc}')
        docs.append(doc)                     # anadimos el archivo a la lista docs

In [ ]:
docs

In [ ]:
len(docs)

`docs` es ahora una lista con las rutas a los documentos PDF. Ahora vamos a iterar a traves de la lista para leerlos usando la funcion `texto_PDF()` que definiremos en el siguiente bloque.

Esta funcion es bastante complicada, pero funciona bien para casi todos los casos en que cualquier usuario tendria que leer un PDF. No es necesario entender todo lo que contiene la funcion dado que algunos de estos comandos son bastante especializados. Para nuestro uso, no la modificaremos y la vamos a utilizar tal como esta.

In [ ]:
def texto_PDF(pdfFile):
    
    # Basado en codigo de http://stackoverflow.com/a/20905381/4955164
    # El ejemplo usa encoding UTF-8. Esto se puede cambiar a otros encodings
    # para textos con caracteres inusuales
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams) #, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    
    return returnedString

Es muy importante notar que el input de `texto_pdf()` **no es la ruta del archivo PDF sino la lectura (en bytes) del archivo**. Para obtener la lectura en bytes, tenemos que abrir los archivos primero. Para esto usaremos una funcion muy frecuente para abrir archivos en Python: `open()` combinado con la palabra clave `with`:

In [ ]:
print(docs[0])

In [ ]:
# Abriremos el primer archivo en docs como ejemplo y lo usaremos con texto_PDF:
ruta_documento = docs[0]
with open(ruta_documento, 'rb') as f:
    texto = texto_PDF(f)

Usaremos `print()` para visualizar el resultado:

In [ ]:
print(texto)

El resultado no es perfecto, pero funciona bastante bien para realizar analisis de textos. Ahora continuaremos con procesar todos los documentos en la lista `docs`. Este proceso podria tomar algo de tiempo en terminarse.

In [ ]:
textos_completos = []

for doc in docs:
    
    with open(doc, 'rb') as f:
        
        print(f'Leyendo documento {doc}...')
        texto = texto_PDF(f)
        textos_completos.append(texto)
        print('\tFinalizado')

In [ ]:
len(textos_completos)

`textos_completos` ahora tiene los textos enteros de cada documento PDF listado en `docs`. Con esto, podemos seguir dando estructura a los textos en un dataframe de Pandas.

In [ ]:
import pandas as pd

In [ ]:
df_textos = pd.DataFrame()

In [ ]:
# Anadiendo la ruta de los documentos como columna
df_textos['archivos'] = docs

In [ ]:
# Anadiendo los textos como columna
df_textos['textos'] = textos_completos

In [ ]:
df_textos

In [ ]:
# Inspeccionando una observacion
df_textos['textos'][3]

Ahora nuestros textos ya tienen una estructura de datos! con esto, podemos anadir nuevas columnas a `df_textos` con caracteristicas sobre los archivos que hemos leido. Para este ejemplo, anadiremos una columna con el numero de caracteres y otra con una variable dummy marcando cuales de los textos contienen las palabras clave "IVA" e "impuestos".

### Numero de caracteres

Para contar el numero de caracteres en cada texto usaremos el metodo `apply.()`. `.apply()` funciona de forma vectorizada, de modo que es mucho mas rapido que aplicar una funcion mediante un bucle. `.apply()` es similar al metodo `.map()` que se usa en Series de pandas, con la diferencia de que `.apply()` funciona en columnas de dataframes (`.map()` no se puede usar en dataframes).

In [ ]:
# Nueva columna
df_textos['n_caracteres'] = df_textos['textos'].apply(len)

In [ ]:
df_textos

### Palabras en el texto

Nuestra siguiente columna sera una *dummy* (un valor que es uno o cero, donde el valor uno indica la presencia de una caracteristica) indicando que textos contienen las palabras clave "IVA" o "impuestos".

Para esto, crearemos una funcion que toma un texto y verifica si las palabras clave estan en el. Luego usaremos `.apply()` para aplicar la funcion de forma vectorizada.

In [ ]:
def palabras_IVA_impuestos(texto):
    
    palabras = ['IVA', 'impuestos', 'Impuestos']
    
    for palabra in palabras:
        
        if palabra in texto:
            
            return 1
    
    return 0

In [ ]:
df_textos['mencion_IVA_impuestos'] = df_textos['textos'].apply(palabras_IVA_impuestos)

In [ ]:
df_textos

De esta forma podemos continuar agregando columnas al dataframe para proceder a analizar los textos. Esto es posible porque los datos ahora estan estructurados.

Por ahora, dejaremos de trabajar con este dataframe para continuar los ejemplos de esta y la proxima sesion utilizando textos mas pequenos en lugar de documentos enteros. Todas las operaciones que veremos a continuacion se pueden aplicar en los textos de `df_textos` o en textos largos.